# 5. The Vector API: tipg

<https://developmentseed.org/tipg>

No geospatial data stack is complete without a mechanism for serving vector features as GeoJSON or for consumption in web maps as vector tiles. **tipg** is the component that fills that niche in eoAPI.

From the tipg README:
> `tipg`, pronounced *T[ee]pg*, is a **Python** package that helps create lightweight OGC **Features** and **Tiles** API with a PostGIS Database backend. The API has been designed for [OGC Features](https://ogcapi.ogc.org/features) and [OGC Tiles](https://ogcapi.ogc.org/tiles/) specifications.

In addition to serving existing features from a pre-defined set of tables in a PostGIS-enabled PostgreSQL database, it can serve features from custom views defined in user-defined PostgreSQL functions.

## 5.1 Configuration

In an eoAPI stack, tipg can be connected to any PostgreSQL database including the existing pgstac database. This is controlled by the `POSTGRES_*` environment variables in the application runtime.

When deploying tipg, you can specify which schemas in your database will be exposed to the tipg API. This is controlled by the `TIPG_DB_SCHEMAS` environment variable.

For the workshop we have created a schema in the pgstac database called `features` that you will be working with. To expose this schema to tipg we set `TIPG_DB_SCHEMAS=["features"]` in the application runtime (see line 178 in [infrastructure/app.py](../infrastructure/app.py)). There is one table that has been pre-loaded (`features.ecoregions`) for the examples in this notebook.

### Additional resources:
- tipg configuration docs: <https://developmentseed.org/tipg/user_guide/configuration>

## 5.2 OGC API - Features
tipg contains an OGC Features API that is interoperable with many existing applications.

<https://ogcapi.ogc.org/features/overview.html>

Each table in the PostgreSQL schema represents a single collection. The list of collections available to tipg can be obtained with a GET request to the `/collections` endpoint

In [1]:
import json
import os

import httpx

tipg_endpoint = os.getenv("TIPG_API_ENDPOINT")

collections_request = httpx.get(f"{tipg_endpoint}/collections")

print(json.dumps(collections_request.json(), indent=2))

{
  "links": [
    {
      "href": "http://tipg:8083/collections",
      "rel": "self",
      "type": "application/json"
    }
  ],
  "numberMatched": 1,
  "numberReturned": 1,
  "collections": [
    {
      "id": "features.ecoregions",
      "title": "features.ecoregions",
      "links": [
        {
          "href": "http://tipg:8083/collections/features.ecoregions",
          "rel": "collection",
          "type": "application/json"
        },
        {
          "href": "http://tipg:8083/collections/features.ecoregions/items",
          "rel": "items",
          "type": "application/geo+json"
        },
        {
          "href": "http://tipg:8083/collections/features.ecoregions/queryables",
          "rel": "queryables",
          "type": "application/schema+json"
        },
        {
          "href": "http://tipg:8083/collections/features.ecoregions/tiles",
          "rel": "data",
          "type": "application/json",
          "title": "Collection TileSets"
        },
       

Each collection has a set of links associated with it including:
- `/collections/{collection_id}/queryables`: describes the fields that can be used for filtering features
- `/collections/{collection_id}/items`: where features can be accessed
- `/collections/{collection_id}/tiles`: list of tile matrix set IDs that are available for tile requests
- `/collections/{collection_id}/tiles/{tileMatrixSetId}`: returns a tilejson for a vector tile layer
- `/collections/{collection_id}/tiles/{tileMatrixSetId}/viewer`: interactive map of the collection

The `/items`, `/tiles/{tileMatrixSetId}`, and `/tiles/{tileMatrixSetId}/viewer` endpoints will all accept field filters in the form of `{queryable}={value}` where `queryable` is one of the fields listed in the `/queryables` response for that collection.

### 5.2.2 /collections/{collection_id/queryables

The `/queryables` endpoint returns a list of fields that can be used to filter features in requests for a collection:

In [2]:
collection_id = "features.ecoregions"

queryables_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/queryables"
)

print(json.dumps(queryables_request.json(), indent=2))

{
  "title": "features.ecoregions",
  "properties": {
    "geom": {
      "$ref": "https://geojson.org/schema/MultiPolygon.json"
    },
    "id": {
      "name": "id",
      "type": "number"
    },
    "na_l1code": {
      "name": "na_l1code",
      "type": "string"
    },
    "na_l1key": {
      "name": "na_l1key",
      "type": "string"
    },
    "na_l1name": {
      "name": "na_l1name",
      "type": "string"
    },
    "na_l2code": {
      "name": "na_l2code",
      "type": "string"
    },
    "na_l2key": {
      "name": "na_l2key",
      "type": "string"
    },
    "na_l2name": {
      "name": "na_l2name",
      "type": "string"
    },
    "na_l3code": {
      "name": "na_l3code",
      "type": "string"
    },
    "na_l3key": {
      "name": "na_l3key",
      "type": "string"
    },
    "na_l3name": {
      "name": "na_l3name",
      "type": "string"
    },
    "shape_area": {
      "name": "shape_area",
      "type": "number"
    },
    "shape_leng": {
      "name": "shape_leng"

### 5.2.2 /collections/{collection_id}/items

The `/items` endpoint for a collection can be used to retrieve paginated lists of features in a number of formats:
  - GeoJSON
  - CSV
  - JSON
  - GeoJSON Sequence
  - NDJSON (new-line-delimited JSON)
  - HTML (for viewing in a browser)

[source](https://github.com/developmentseed/tipg/blob/1a2e5eb6816d51f97ae1d5bbc1d1e952d996987b/tipg/factory.py#L757-L762)

Try retrieving a page of results from the `features.terrestrial_ecoregions` collection. This will return a GeoJSON FeatureCollection with two features (`limit=2`):

In [3]:
geojson_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/items",
    params={"f": "geojson", "limit": 2},
)
geojson_response = geojson_request.json()
print(json.dumps(geojson_response, indent=2))

{
  "type": "FeatureCollection",
  "id": "features.ecoregions",
  "title": "features.ecoregions",
  "description": "features.ecoregions",
  "numberMatched": 2548,
  "numberReturned": 2,
  "links": [
    {
      "title": "Collection",
      "href": "http://tipg:8083/collections/features.ecoregions",
      "rel": "collection",
      "type": "application/json"
    },
    {
      "title": "Items",
      "href": "http://tipg:8083/collections/features.ecoregions/items?f=geojson&limit=2",
      "rel": "self",
      "type": "application/geo+json"
    },
    {
      "href": "http://tipg:8083/collections/features.ecoregions/items?f=geojson&limit=2&offset=2",
      "rel": "next",
      "type": "application/geo+json",
      "title": "Next page"
    }
  ],
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "MultiPolygon",
        "coordinates": [
          [
            [
              [
                -81.284387025,
                75.650554212
              ],
   

You can request a sequence of GeoJSON features separated by new lines with `f=geojsonseq`

In [4]:
geojsonseq_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/items",
    params={"f": "geojsonseq", "limit": 2},
)

print(geojsonseq_request.text)

{"type":"Feature","geometry":{"type":"MultiPolygon","coordinates":[[[[-81.284387025,75.650554212],[-81.011775093,75.627851846],[-80.969108863,75.627073768],[-80.858757155,75.630592482],[-80.793048729,75.634307935],[-80.637982058,75.649422886],[-80.5369486,75.654593215],[-80.509016767,75.653187825],[-80.445786235,75.638602836],[-80.309806437,75.634001693],[-80.282139476,75.631739425],[-80.254837391,75.62722222],[-80.068107935,75.579888831],[-80.024958206,75.540527808],[-80.043131086,75.525139513],[-80.179309219,75.496839667],[-80.257105812,75.485780393],[-80.365236415,75.480050727],[-80.376314957,75.476583345],[-80.380465856,75.472015719],[-80.369735142,75.459035277],[-80.351434942,75.456618732],[-80.269131357,75.458260017],[-79.98348459,75.478892431],[-79.90159185,75.478622895],[-79.702778688,75.468949527],[-79.631591318,75.456879316],[-79.651011312,75.448529197],[-79.675670113,75.442183662],[-79.680666282,75.432781874],[-79.630697413,75.400839975],[-79.5083147,75.371337673],[-79.47532

You can apply a filter using the fields returned in the `/queryables` endpoint for a collection:

In [5]:
filtered_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/items",
    params={
        "na_l2name": "MEDITERRANEAN CALIFORNIA",
        "f": "geojson",
        "limit": 2,
    },
)

filtered_response = filtered_request.json()
print(
    f"{filtered_response['numberMatched']} features match this filtered request",
    f"out of {geojson_response['numberMatched']} features in the entire collection",
)

45 features match this filtered request out of 2548 features in the entire collection


In addition to field-based filters, you can use other standard filter mechanisms:
- `ids`: limit to a comma-separated list of feature ids
- `bbox`: filter by bounding box
- `datetime`: filter by datetime (use with `datetime-column` parameter)
- `filter`: apply a CQL2 filter (use with `filter-lang` parameter set to cql2-text or cql2-json) for more complex filter operations

In [6]:
# filter by bounding box
bbox_filtered_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/items",
    params={"bbox": "-77,39,-76,40", "f": "geojson", "limit": 2},
)

bbox_filtered_response = bbox_filtered_request.json()
print(
    f"{bbox_filtered_response['numberMatched']} features match this filtered request",
    f"out of {geojson_response['numberMatched']} features in the entire collection",
)

20 features match this filtered request out of 2548 features in the entire collection


tipg also comes with a convenient HTML response type which makes it possible to interact with the endpoints in your browser. The returned geojson features from a `/items` request will be displayed in a map!

In [10]:
from IPython.display import IFrame

bbox_filtered_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/items",
    params={
        "bbox": "-77,39,-76,40",
        "f": "html",
    },
)

local_url = str(bbox_filtered_request.url).replace("tipg", "localhost")

IFrame(
    local_url,
    width=1200,
    height=800,
)

Here is a view of the full API docs for the `/collections/{collection_id}/items` endpoint:

In [11]:
local_tipg_endpoint = tipg_endpoint.replace("tipg", "localhost")
IFrame(
    f"{local_tipg_endpoint}/api.html#OGC Features API/items_collections__collectionId__items_get",
    width=1200,
    height=800,
)

## 5.3 OGC API - Tiles

tipg also serves an OGC Tiles API for vector tiles.

<https://ogcapi.ogc.org/tiles/>

The Tiles API works exactly like the Features API but instead of taking requests for entire features it accepts requests for XYZ vector tiles that are very useful for streaming vector data into map client applications. This is useful because it will becomes impracticalimpossible to stream all of a collection's features into a map application as a geojson - tipg moves the simplification and filtering operations up to the PostGIS database and returns the minimum required data to the map client.

### 5.3.1 /collections/{collection_id/tiles/{tileMatrixSetId}/tilejson.json

The `tilejson` endpoint is the most useful for adding vector tile layers to a map application. The response contains information about the available fields (which can be used for styling the vector tiles), the full collection extent, and the XYZ tile url that can be loaded as a layer in a map.

In [12]:
tilejson_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/tiles/WebMercatorQuad/tilejson.json",
)
tilejson_response = tilejson_request.json()
print(json.dumps(tilejson_response, indent=2))

{
  "tilejson": "3.0.0",
  "name": "features.ecoregions",
  "version": "1.0.0",
  "scheme": "xyz",
  "tiles": [
    "http://tipg:8083/collections/features.ecoregions/tiles/WebMercatorQuad/{z}/{x}/{y}"
  ],
  "vector_layers": [
    {
      "id": "default",
      "fields": {
        "id": "number",
        "na_l1code": "string",
        "na_l1key": "string",
        "na_l1name": "string",
        "na_l2code": "string",
        "na_l2key": "string",
        "na_l2name": "string",
        "na_l3code": "string",
        "na_l3key": "string",
        "na_l3name": "string",
        "shape_area": "number",
        "shape_leng": "number"
      },
      "minzoom": 0,
      "maxzoom": 22
    }
  ],
  "minzoom": 0,
  "maxzoom": 22,
  "bounds": [
    -179.13073409090276,
    14.515827106773095,
    -52.661774883495006,
    83.1104480492809
  ],
  "center": [
    -115.89625448719889,
    48.813137578026996,
    0
  ]
}


All of the same rules for queryables and query parameters from the `/items` endpoint apply to the `/tiles` endpoints, too. The query parameters will be tacked onto the end of the XYZ tile URL:

In [13]:
filtered_tilejson_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/tiles/WebMercatorQuad/tilejson.json",
    params={
        "eco_name": "Northern Mesoamerican Pacific mangroves",
    },
)
filtered_tilejson_response = filtered_tilejson_request.json()
print(json.dumps(filtered_tilejson_response, indent=2))

{
  "tilejson": "3.0.0",
  "name": "features.ecoregions",
  "version": "1.0.0",
  "scheme": "xyz",
  "tiles": [
    "http://tipg:8083/collections/features.ecoregions/tiles/WebMercatorQuad/{z}/{x}/{y}?eco_name=Northern+Mesoamerican+Pacific+mangroves"
  ],
  "vector_layers": [
    {
      "id": "default",
      "fields": {
        "id": "number",
        "na_l1code": "string",
        "na_l1key": "string",
        "na_l1name": "string",
        "na_l2code": "string",
        "na_l2key": "string",
        "na_l2name": "string",
        "na_l3code": "string",
        "na_l3key": "string",
        "na_l3name": "string",
        "shape_area": "number",
        "shape_leng": "number"
      },
      "minzoom": 0,
      "maxzoom": 22
    }
  ],
  "minzoom": 0,
  "maxzoom": 22,
  "bounds": [
    -179.13073409090276,
    14.515827106773095,
    -52.661774883495006,
    83.1104480492809
  ],
  "center": [
    -115.89625448719889,
    48.813137578026996,
    0
  ]
}


### 5.3.1 /collections/{collection_id/tiles/{tileMatrixSetId}/viewer

For a quick demonstration of how vector tiles enable visualization of massive feature collections, check out this map of the `terrestrial_ecoregions` table that lives in our database. It has 14,000+ features which we would never dream of downloading to view in a web map. Instead, we let our map client make requests for simplified features for each XYZ tile as we explore the map.

In [16]:
viewer_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/tiles/WebMercatorQuad/viewer",
)

IFrame(
    str(viewer_request.url).replace("tipg", "localhost"),
    width=1200,
    height=800,
)

You can apply a field-based filter to limit the features to a subset of the full collection:

In [18]:
filtered_viewer_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/tiles/WebMercatorQuad/viewer",
    params={
        "na_l2name": "MEDITERRANEAN CALIFORNIA",
    },
)

IFrame(
    str(filtered_viewer_request.url).replace("tipg", "localhost"),
    width=1200,
    height=800,
)